# [2] Algorithms: String Matching

## Brute Force String Matching

In [55]:
def string_matching(pattern, text):
    count = 0
    for i in range(0, len(text) - len(pattern) + 1):
        match = 0
        for j, p_char in enumerate(pattern):
            count += 1
            if text[i+match] == p_char:
                    match += 1
            else:
                break
            # print(count)
            if len(pattern) == match:
                return True           
    return False
   
string_matching("mira", "amamiraaaaaaa")

True

## Brute Force String Matching aus dem TGI02-Skript

In [56]:
def string_matching2(p, S, print_flag = True):
    i, j, count = 0, 0, 0
    if print_flag: print(f"\033[94m{S}\033[0m")
    while(i<len(S) and j<len(p)):
        if print_flag:
            print(f"{(i-j)*'_'}{p[:j]}\033[1m{p[j]}\033[0m{p[j+1:]}{(len(S)-len(p)-(i-j))*'_'}")
        count += 1
        if p[j] == S[i]:
            j += 1
            i += 1
        else:
            #if i > (len(S)-len(p)-1) and j == 0: return False
            i = i - j + 1
            j = 0
            
    if j==len(p): return True
    if print_flag: print(count)
    return False
    
string_matching2("mira", "aaaaaaamamir")

aaaaaaamamir
mira________
_mira_______
__mira______
___mira_____
____mira____
_____mira___
______mira__
_______mira_
_______mira_
________mira
_________mira
_________mira
_________mira
13


False

# KMP binary Matching  Algorithm - TGI02 Script

In [6]:
def calc_next_array(p):
    """calculate the shift array or lps array"""
    n = p.bit_length()
    next = []
    for i in range(n):
        next_value = 0
        if i != 0:
            # p auf i+1 te STelle kuerzen
            p_short = [int(x) for x in bin(p)[2:i+1+2]]  # +1 da stop exklusive +2 wg 0b
            # p_new = p' berechnen, das Mismatch an Stelle i erzeugt
            p_new = p_short.copy()
            p_new[i] ^= 1  # XOR 1 switched genau 0 und 1

            m = i + 1 # Laenge des gekuerzten p bzw p'
            for j, digit in enumerate(p_short, 1):
                condition = all([p_short[k] == p_new[m - j +k] for k in range(0,j)])
                next_value = next_value if not condition else j
        next.append(next_value)
    return next

p = 0b10100111
calc_next_array(p)

[0, 1, 0, 1, 3, 0, 2, 2]

In [58]:
def matching_KMP2(pattern, scantext):
    i, j = 0, 0
    next = calc_next_array(pattern)
    print("next_array: ", next)
    scan_string = str(bin(scantext))[2:]
    p = str(bin(pattern))[2:]
    print(scan_string)
    while i < len(scan_string) and j < len(p):
        print(f"{(i-j)*'_'}{p[:j]}\033[1m{p[j]}\033[0m{p[j+1:]}{(len(scan_string)-len(p)-(i-j))*'_'}")
        if p[j] == scan_string[i]:
            if j == len(p)-1: return True
            j += 1
        else:
            j = next[j]
        i += 1
    return False

p = 0b101001

matching_KMP2(p, 0b10101010010)

next_array:  [0, 1, 0, 1, 3, 0]
10101010010
101001_____
101001_____
101001_____
101001_____
101001_____
__101001___
__101001___
____101001_
____101001_
____101001_


True

In [67]:
def matchingKMP(p, S):
    """ TGI02"""
    i, j = 0, 0
    next_array = calc_next_array(int(p,2))
    print(next_array)
    print(S)
    while (i < len(S) and j < len(p)):
        print(f"{(i-j)*'_'}{p[:j]}\033[1m{p[j]}\033[0m{p[j+1:]}{(len(S)-len(p)-(i-j))*'_'}")
        # println "i:" + i + " j:" + j
        if (p[j] == S[i]):
            i+=1
            j+=1
        
        else:
            j = next_array[j]
            i+=1
    
    if j == len(p): return "vorhanden"  
    return "nicht vorhanden"

p = "101001"
matchingKMP(p, "10101010010")

[0, 1, 0, 1, 3, 0]
10101010010
101001_____
101001_____
101001_____
101001_____
101001_____
__101001___
__101001___
____101001_
____101001_
____101001_


'vorhanden'

In [60]:
p = "011"
bin(int(p,2))

'0b11'

## KMP String Matching Algorithm - not binary!

In [7]:
def compute_next_array(pattern):
    n = len(pattern)
    pointer = 0
    i = 1
    next_array = [0]*len(pattern)
    while i < n:
        if pattern[i] == pattern[pointer]:
            pointer += 1
            next_array[i] = pointer
            i += 1
        else:
            if pointer != 0:
                pointer = next_array[pointer-1]
            else:
                next_array[i] = 0
                i += 1    
    return next_array

def string_matching_kmp(pattern, text):
    next_array = compute_next_array(pattern)
    count = 0
    i = 0
    while i < len(text) - len(pattern) + 1:
        match = 0
        for j, p_char in enumerate(pattern):
            count += 1
            if text[i+match] == p_char:
                match += 1   
            else:
                print(match, next_array[match])
                i += 1 # (next_array[match])
                break
           
        print(count)        
        if len(pattern) == match:
            return True
         
    return False
compute_next_array("onions")
# string_matching_kmp("mira", "amamiraaaaaaa")

[0, 0, 0, 1, 2, 0]

In [3]:
def compute_lps(pattern: str) -> list[int]:
    # Longest Proper Prefix that is suffix array
    lps = [0] * len(pattern)

    prefi = 0
    for i in range(1, len(pattern)):
        
        # Phase 3: roll the prefix pointer back until match or 
        # beginning of pattern is reached
        while prefi and pattern[i] != pattern[prefi]:
            prefi = lps[prefi - 1]

        # Phase 2: if match, record the LSP for the current `i`
        # and move prefix pointer
        if pattern[prefi] == pattern[i]:
            prefi += 1
            lps[i] = prefi

        # Phase 1: is implicit here because of the for loop and 
        # conditions considered above

    return lps


compute_lps("onions")

[0, 0, 0, 1, 2, 0]

# Boyer-Moore String Matching
pattern wird immer von hinten angefangen zu überprüfen, bei mismatch gemaess shift table von links nach rechts geshifted

In [ ]:
def generate_shift_table(pattern):
    n = len(pattern)
    skip_list = {} # creating a dictionary
    for i, letter in enumerate(pattern) :
        skip_list[letter] = n-i-1 #max(1, n - i - 1)
        # key will be the character, value will be the number of shifts
    return skip_list

generate_shift_table("olgo")

{'o': 0, 'l': 2, 'g': 1}

In [4]:
def boyer_moore_matching(pattern, scantext, print_flag = True):
    n = len(pattern)
    i = n-1
    j = i
    match = 0
    nextT = generate_shift_table(pattern)
    if print_flag: 
        print("shift array: ", nextT)
        print(scantext)
    
    while i < len(scantext) and j >= 0:
        # print(i, j)
        if print_flag: 
            print_string = f"{(i-j)*'_'}{pattern[:j]}"\
                f"\033[1m{pattern[j]}\033[0m"\
                f"{pattern[j+1:]}"\
                f"{(len(scantext)-len(pattern)-(i-j))*'_'}"
            print(print_string)
        if pattern[j] == scantext[i]:
            if j == 0: return True
            j -= 1
            i -= 1
            match += 1
        else: 
            print(nextT.get(scantext[i], n))
            i +=  nextT.get(scantext[i], n) # pattern[j]
            j = n-1
            match = 0
    return False

p = "abcdef"
s = "aababcabcdabcdeabcdef"
print(len(p), len(s))
boyer_moore_matching(p, s)

6 21
shift array:  {'a': 5, 'b': 4, 'c': 3, 'd': 2, 'e': 1, 'f': 0}
aababcabcdabcdeabcdef
abcdef_______________
3
___abcdef____________
3
______abcdef_________
4
__________abcdef_____
5
_______________abcdef
_______________abcdef
_______________abcdef
_______________abcdef
_______________abcdef
_______________abcdef


True

https://cmps-people.ok.ubc.ca/ylucet/DS/BoyerMoore.html

In [16]:
boyer_moore_matching("hing", "This is a searching example")

shift array:  {'h': 3, 'i': 2, 'n': 1, 'g': 0}
This is a searching example
hing_______________________
____hing___________________
________hing_______________
____________hing___________
_______________hing________
_______________hing________
_______________hing________
_______________hing________


True

# Rabin-Karp-Matching

In [63]:
def calc_hash(x):
    return int(x) % 11


def matching_Rabin_Karp(p, S):
    pattern_hash = calc_hash(p)
    
    for si in range(len(S) -len(p)):
        tmp = S[si:si + len(p)]
        
        scan_hash = calc_hash(tmp)
        if pattern_hash == scan_hash:
            if string_matching2(p, tmp, print_flag = True): return True
    return False

pattern = "841"
scantext = "852467384194"
matching_Rabin_Karp(pattern, scantext)


852
841
841
_841
__841
4
467
841
_841
__841
3
841
841
841
841


True

In [28]:
import math
math.log(2,2)

1.0

In [19]:
def nextT(j):
    if (j == "A"): return 3
    if (j == "l"): return 2
    if (j == "g"): return 1        
    return 4

def matchingBoyerMoore(p, S):
    si = len(p)-1 # .Size()
    pi = si
    print(S)
    stop=0
    while (si < len(S)):
        print(pi, si)
        print_string = f"{(si-pi)*'_'}{p[:pi]}"\
            f"\033[1m{p[pi]}\033[0m"\
            f"{p[pi+1:]}"\
            f"{(len(S)-len(p)-(si-pi))*'_'}"
        print(S)
        print(print_string)
        if (p[pi] == S[si]):
            if (pi == 0): 
                print(stop)
                return True
            pi = pi - 1
            si = si - 1
        else:
            print(nextT(S[si]))
            #si = si + nextT(S[si])
            si = si + (len(p) -pi -1)+nextT(S[si+len(p)-1-pi])
            pi = len(p)-1
        stop += 1
        if stop > 30: return False
    return False
matchingBoyerMoore("Algo", "glgoA")
#matchingBoyerMoore("Algo", "gfglzugkgooehjsehbAlgkpodfdovAlgoooo")
#matchingBoyerMoore("Algo", "coArgoAlgo")

glgoA
3 3
glgoA
Algo_
2 2
glgoA
Algo_
1 1
glgoA
Algo_
0 0
glgoA
Algo_
1


False

In [3]:
def back(j):
    if (j == "A"): return 4
    if (j == "l"): return 3
    if (j == "g"): return 2
    return 1


def matchingBoyerMoore(p, S):
    si = len(p)-1 # .Size()
    pi = si
    print(S)
    stop=0
    while (si < len(S)):
        print(pi, si)
        print_string = f"{(si-pi)*'_'}{p[:pi]}"\
            f"\033[1m{p[pi]}\033[0m"\
            f"{p[pi+1:]}"\
            f"{(len(S)-len(p)-(si-pi))*'_'}"
        print(S)
        print(print_string)
        if (p[pi] == S[si]):
            if (pi == 0): 
                print(stop)
                return True
            pi = pi - 1
            si = si - 1

        else:
            print(pi, back(S[si]))
            #si = si + (len(p) -pi -1)+back(S[si+1])
            si = si + (len(p) -pi -1)+back(S[si+len(p)-1-pi])
            #si = si  + back(S[si])
            pi = len(p)-1
            
            #if pi <0: 
            #    pi = len(p)-1
            #    si += len(
            #si += len(p)
            #si = si - back(S[si])
        stop += 1
        if stop >30: return False
    return False
#matchingBoyerMoore("Algo", "coArgoAlgo")
matchingBoyerMoore("Algo", "gfglzugkgooehjsehbAlgkpodfdovAlgoooo")

gfglzugkgooehjsehbAlgkpodfdovAlgoooo
3 3
gfglzugkgooehjsehbAlgkpodfdovAlgoooo
Algo________________________________
3 3
3 6
gfglzugkgooehjsehbAlgkpodfdovAlgoooo
___Algo_____________________________
3 2
3 8
gfglzugkgooehjsehbAlgkpodfdovAlgoooo
_____Algo___________________________
3 2
3 10
gfglzugkgooehjsehbAlgkpodfdovAlgoooo
_______Algo_________________________
2 9
gfglzugkgooehjsehbAlgkpodfdovAlgoooo
_______Algo_________________________
2 1
3 11
gfglzugkgooehjsehbAlgkpodfdovAlgoooo
________Algo________________________
3 1
3 12
gfglzugkgooehjsehbAlgkpodfdovAlgoooo
_________Algo_______________________
3 1
3 13
gfglzugkgooehjsehbAlgkpodfdovAlgoooo
__________Algo______________________
3 1
3 14
gfglzugkgooehjsehbAlgkpodfdovAlgoooo
___________Algo_____________________
3 1
3 15
gfglzugkgooehjsehbAlgkpodfdovAlgoooo
____________Algo____________________
3 1
3 16
gfglzugkgooehjsehbAlgkpodfdovAlgoooo
_____________Algo___________________
3 1
3 17
gfglzugkgooehjsehbAlgkpodfdovAlgoooo
______________Al

False

In [13]:
def define_back(w):
    D = {}
    for i in range(0, len(w)-1):
        D[w[i]]= len(w) - i
    
    return D

print(define_back("Algo"))

{'A': 4, 'l': 3, 'g': 2}
